#### Adding logic for incorporating Historical messages

### 06-02-2025

In [1]:
## Here also we are using Opensource models
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq

groq_api_key = os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model="llama-3.3-70b-specdec")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x109becc70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x109bed840>, model_name='llama-3.3-70b-specdec', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [2]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
from langchain.embeddings import HuggingFaceEmbeddings
# Here we are using "all-MiniLM-L6-v2" because sentence embedding model that can be used for tasks like semantic search, clustering, and information retrieval 
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/var/folders/w7/n9_6cd9j7w73pds6f3kyslfm0000gp/T/ipykernel_3295/544177755.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/Users/surya/Desktop/Langchain/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
## 1. Loading, chunk and index the contents of the blog to create a retriver
import bs4
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content","post-title","post-header")
        )
    ),
)

docs= loader.load()
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistake

In [5]:
text_splitter=RecursiveCharacterTextSplitter (chunk_size=1000, chunk_overlap=200)
splits=text_splitter.split_documents (docs)
vectorstore=Chroma.from_documents (documents=splits, embedding=embeddings)
retriever=vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x16b1be680>, search_kwargs={})

In [6]:
## Prompt Template
system_prompt=(
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human","{input}"),
    ]
)

In [7]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [8]:
response = rag_chain.invoke({"input":"What is Self-Reflection"})
print(response)

{'input': 'What is Self-Reflection', 'context': [Document(id='d438b5e2-633d-4386-84c7-e3a6393b6042', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 3. Illustration of the Reflexion framework. (Image source: Shinn & Labash, 2023)\nThe heuristic function determines when the trajectory is inefficient or contains hallucination and should be stopped. Inefficient planning refers to trajectories that take too long without success. Hallucination is defined as encountering a sequence of consecutive identical actions that lead to the same observation in the environment.\nSelf-reflection is created by showing two-shot examples to LLM and each example is a pair of (failed trajectory, ideal reflection for guiding future changes in the plan). Then reflections are added into the agent’s working memory, up to three, to be used as context for querying LLM.'), Document(id='2011c32e-3232-4e55-9717-c81bceba02ab', metadata={'source': 'https://lilianweng.gith

In [9]:
response

{'input': 'What is Self-Reflection',
 'context': [Document(id='d438b5e2-633d-4386-84c7-e3a6393b6042', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 3. Illustration of the Reflexion framework. (Image source: Shinn & Labash, 2023)\nThe heuristic function determines when the trajectory is inefficient or contains hallucination and should be stopped. Inefficient planning refers to trajectories that take too long without success. Hallucination is defined as encountering a sequence of consecutive identical actions that lead to the same observation in the environment.\nSelf-reflection is created by showing two-shot examples to LLM and each example is a pair of (failed trajectory, ideal reflection for guiding future changes in the plan). Then reflections are added into the agent’s working memory, up to three, to be used as context for querying LLM.'),
  Document(id='2011c32e-3232-4e55-9717-c81bceba02ab', metadata={'source': 'https://lilianweng.g

In [10]:
response['answer']

"Self-reflection is a vital aspect that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It involves creating reflections by showing examples to a large language model (LLM) and adding them to the agent's working memory to guide future changes in the plan. This process enables agents to learn from their experiences and adapt to new situations."

In [11]:
rag_chain.invoke({"input":"How do we achieve it?"})
#response1
#response1['answer']

{'input': 'How do we achieve it?',
 'context': [Document(id='aeceee80-a23b-4e2b-b313-f8694cbd1a57', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.\nTask decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.'),
  Document(id='cfabe67e-8f4f-453a-8ae2-a77487799b33', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overvi

#### Adding Chat History

In [12]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompts = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood, "
    "without the chat history. Do NOT answer the question,"
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompts),
            MessagesPlaceholder("chat_history"), 
            ("human", "{input}"),
        ]
)

In [13]:
history_aware_retriever = create_history_aware_retriever(llm,retriever,contextualize_q_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x16b1be680>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk'

In [14]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [15]:
question_answer_chain = create_stuff_documents_chain(llm,qa_prompt)
rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)

In [16]:
from langchain_core.messages import AIMessage,HumanMessage
chat_history = []
question = "What is Self-Reflection?"
response1 = rag_chain.invoke({"input":question,"chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1["answer"])
    ]
)
question2 = "Explain abou it more"
response2 = rag_chain.invoke({"input":question,"chat_history":chat_history})

print(response2['answer'])

Self-reflection is a process that allows autonomous agents to improve by refining past decisions and correcting mistakes. It involves analyzing past actions and using that knowledge to guide future behavior. This helps agents learn from their experiences and adapt to new situations.


In [17]:
chat_history

[HumanMessage(content='What is Self-Reflection?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Self-reflection is a vital aspect that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It involves showing two-shot examples to a Large Language Model (LLM) and adding reflections into the agent's working memory to guide future changes in the plan. This process enables agents to learn from their experiences and adapt to new situations.", additional_kwargs={}, response_metadata={})]

In [18]:
## ANother
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store= {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)

In [19]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
        },     ## Constructs a key "abc123" in store
)["answer"]

"Task decomposition is the process of breaking down a complex task into smaller, simpler steps. It involves transforming big tasks into multiple manageable tasks to facilitate interpretation of the model's thinking process. This can be achieved through various methods, including chain of thought, tree of thoughts, and simple prompting techniques."

In [20]:
conversational_rag_chain.invoke(
    {"input": "What are the common ways of doing it?"},
    config={
        "configurable": {"session_id": "abc123"}
        },     ## Constructs a key "abc123" in store
)["answer"]

'Task decomposition can be done in three common ways: (1) by using a Large Language Model (LLM) with simple prompting, (2) by using task-specific instructions, or (3) with human inputs. These methods help to break down complex tasks into smaller, more manageable steps.'

In [22]:
conversational_rag_chain.invoke(
    {"input": "Types of Memories?"},
    config={
        "configurable": {"session_id": "abc123"}
        },     ## Constructs a key "abc123" in store
)["answer"]

'There are several types of memories, including: \n1. Sensory Memory: retains impressions of sensory information for a short period.\n2. Short-Term Memory (STM): stores information for a short duration (20-30 seconds) and has limited capacity.\n3. Long-Term Memory (LTM): has unlimited storage capacity and can store information for a long time, with two subtypes: \n   - Explicit/Declarative Memory: conscious recall of facts and events.\n   - Implicit/Procedural Memory: unconscious recall of skills and routines.'